## LangChain

- One of the earliest and most widely used
- Huge ecosystem
- Integrates memory, tools, chains, retrieval, agents

https://www.langchain.com/langchain


```
pip install -U langchain
```

In [ ]:
import os
import requests
from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain.tools import tool

load_dotenv()
aval_api_key=os.getenv("AVALAI_API_KEY")
open_weather_api_key=os.getenv("OPENWEATHER_API_KEY")


In [ ]:
@tool
def get_weather(city: str, unit: str = "metric"):
    """
    Extracts the weather of a city from openweathermap

    :param city: name of city
    :type city: str
    :param unit: unit, defaults to "metric"
    :type unit: str, optional
    :return: return a json containing the city name, temp and a desc
    :rtype: json
    """
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={open_weather_api_key}&units={unit}"
    resp = requests.get(url)
    data = resp.json()
    return {
        "city": city,
        "temp": data["main"]["temp"],
        "desc": data["weather"][0]["description"]
    }

In [ ]:
agent = create_agent(
    model="gpt-4o-mini",
    
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

# Run the agent
result = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in Tehran"}]}
)

In [ ]:
result

In [ ]:
result["messages"][-1].pretty_print()

## Memory

In [ ]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver  


agent = create_agent(
    "gpt-4o-mini",
    tools=[get_weather],
    checkpointer=InMemorySaver(),  
)

agent.invoke(
    {"messages": [{"role": "user", "content": "Hi! My name is Ali."}]},
    {"configurable": {"thread_id": "1"}},  
)

In [ ]:
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in Tehran"}]},
    {"configurable": {"thread_id": "1"}}
)

In [ ]:
final_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in Shiraz tomorrow"}]},
    {"configurable": {"thread_id": "1"}}
)

In [ ]:
agent.invoke(
    {"messages": [{"role": "user", "content": "Do you know my name?"}]},
    {"configurable": {"thread_id": "1"}}
)